In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Colab Notebooks/kobo2020'
%ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/kobo2020
AttentionSeq2seq.pkl  GoogleNews-vectors-negative300.bin
attention_seq2seq.py  negative_sampling_layer.py
BetterRnnlm.pkl       neko.txt
better_rnnlm.py       nlp100_chap9
cbow_params.pkl       nlp100_chap9.pkl
cbow.py               nlp100_chap9.txt
ch04/                 __pycache__/
ch06/                 PyTorchTutorial1.ipynb
ch07/                 repo10.ipynb
ch07＿pra.ipynb       repo6.ipynb
ch08/          

In [5]:
#83
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence
word2id_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/nlp100_chap9'#todo 80.pyで保存したファイルパスを指定
import numpy as np
word2id = pickle.load(open(word2id_path, "rb"))
print(word2id)#IDと単語の辞書列型 {単語:ID}

# word_listの単語リストをIDリストに変換する関数
def get_ids(word_list):
    ids = []
    for word in word_list:
        if word in word2id:#todo wordがword2idのキーに含まれる場合，
            ids.append(word2id[word])#todo そのIDをidsにappend
        else:
            ids.append(0)#todo wordがword2idのキーに含まれない場合，0をidsにappend
    return ids

def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    ids_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()#改行を削除
            line = line.split('\t')#todo lineからタイトルを取得
            title = line[1]
            word_list =  title.split(" ")#todo titleを単語に分割
            ids = get_ids(word_list)
            ids = np.array(ids)
            ids_tensor = torch.from_numpy(ids)#todo idsをtensorに変換
            ids_list.append(ids_tensor)#todo ids_tensorをids_listにappend


            category = line[0]#todo lineからカテゴリーを取得
            label = category2num[line[0]]#todo category2numでカテゴリーをラベルに変換
            label_list.append(label)#todo labelをlabel_listにappend

    labels = torch.from_numpy(np.array(label_list))#todo label_listをtensorに変換
    return ids_list, labels

#分類器
class ClassifierRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(ClassifierRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim)#todo 単語埋め込み層    in : x out : vocab_size x embedding_dim
        self.rnn =nn.LSTM(embedding_dim,hidden_dim)#todo LSTM層  in : vocab_size x embedding_dim  out : 
        self.linear = nn.Linear(hidden_dim,target_size)#todo 線形層  in : 系列x隠れ層　out : 系列x出力ラベル層

    def forward(self, x, x_len):  
        embed_x = self.embedding.forward(x)#todo xを単語埋め込み

        packed_x = pack_padded_sequence(embed_x, x_len, batch_first=True, enforce_sorted=False) #todo embed_xとx_lenを用いてPackedSequence型のデータに変換 
        _, (h, _) = self.rnn(packed_x)#todo LSTM層に入力
        output = self.liner(_)#ltodo 線形層に入力

        output = squeeze(output,0)#todo squeeze()を使用し，outputの形状が[1, batch_size, target_size]となっているものを[batch_size, target_size]に変換

        #print(output.size()) #確認用(確認する場合はコメントアウトを解除)
        return output

class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.length =len(data)#todo dataに含まれる各ID系列の長さのリスト
        self.label = label

        self.pad_data = pad_sequence(data, batch_first=True)#todo dataをpadding

    def __len__(self):
        return len(self.pad_data)

    def __getitem__(self, idx):
        return self.pad_data,self.length,self.label#todo idxに対応するpad_data, length, labelの3つを返り値とする



emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

vocab_size = max(word2id.values()) + 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path =  '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/train.txt'#todo 学習用ファイルパス
train_x, train_y = get_data(train_path)
valid_path = '/content/gdrive/My Drive/Colab Notebooks/kobo2020/data70/test.txt'#todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)


dataset = RnnDataset(train_x, train_y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = ClassifierRNN(vocab_size, emb_dim, hidden_dim, target_size)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, data_len, target in train_loader:
        # GPUへ
        data = data.to(device)
        data_len = data_len.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        pred = model.forward(data,data_len)#todo modelで予測
        loss = loss_fn(pred,target)#todo lossを計算

        loss.backward()
        optimizer.step()

        total_loss += loss#todo lossをtotal_lossに加算
        correct_num +=  (pred == target).sum().item()#todo 予測の正解数をカウント

    batch_loss = total_loss/len_loader #バッチごとの平均ロス
    acc = correct_num/len_loader#todo 正解率
    return batch_loss, acc

# validデータはそのままGPUへ
valid_x_len = []
for i in range(len(valid_x)):
  valid_x_len.append(valid_x[i].shape[0])#todo valid_xの各要素の長さのリスト
valid_x_len = np.array(valid_x_len)
valid_x_len = torch.from_numpy(valid_x_len)
print(type(valid_x_len))

valid_pad_x = pad_sequence(valid_x, batch_first=True)#todo valid_xをpadding

# GPUへ
valid_x_len = valid_x_len.to(device)
valid_pad_x = valid_pad_x.to(device)
valid_y = valid_y.to(device)


def evaluation(model, data, data_len, target):
    model.eval()
    with torch.no_grad():
        pred = model.forward(data.type(torch.FloatTensor))#todo 予測
        loss = loss_fn(pred,target)#todo 損失を計算
        acc = torch.sum(pred_labels == target).item()/len(pred_labels)#todo 正解率を計算
    return loss.item(), acc

len_train = len(train_y)

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)
    valid_loss, valid_acc = evaluation(model, valid_pad_x, valid_x_len, valid_y)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

{'UPDATE': 1, '1-BioDelivery,': 0, 'Endo': 2, 'painkiller': 3, 'effective': 4, 'in': 5, 'study': 6, 'Southern': 7, 'and': 8, 'Turner': 0, 'to': 9, 'Buy': 10, 'Macho': 0, 'Springs': 0, 'From': 11, 'First': 12, 'Solar': 13, 'Twitter': 14, 'forced': 15, 'shut': 16, 'down': 17, 'Tweetdeck': 0, 'app': 18, 'amid': 19, 'major': 20, 'security': 21, 'alert': 22, 'JD.com': 23, 'Goes': 24, 'Public,': 0, 'China': 25, 'Has': 26, 'a': 27, 'New': 28, 'Internet': 29, 'Billionaire': 30, 'Amazon': 31, 'Looks': 32, 'Like': 33, "It's": 34, 'Creating': 35, 'A': 36, 'Netflix': 37, 'For': 38, 'Books': 39, 'Cancer': 40, 'Type': 41, 'May': 42, 'Be': 43, 'Linked': 44, 'With': 45, 'Socioeconomic': 0, 'Status': 46, 'Leaked': 47, 'iPhone': 48, '6': 49, 'Photos': 50, 'Appear': 51, 'To': 52, 'Show': 53, 'Design': 54, 'In': 55, 'All': 56, 'Its': 57, 'Thin,': 0, 'Rounded': 0, '...': 58, 'Hess': 59, 'deal,': 60, 'Marathon': 61, 'fuel': 62, 'traders': 63, 'gain': 64, 'big': 65, 'E.Coast': 0, 'foothold': 66, 'Mel': 67, '

RuntimeError: ignored